In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("SEOKDONG/llama3.0_korean_v1.0_sft")
model = AutoModelForCausalLM.from_pretrained("SEOKDONG/llama3.0_korean_v1.0_sft")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


input_text = """'프로야구[UNK]롯[UNK][UNK][UNK][UNK][UNK]광주 경기 [UNK]천취소' 이 뉴스 헤드라인에는 '[UNK]'로 노이즈가 있어. '[UNK]'에 들어갈 글자를 예측해서 원본 뉴스 헤드라인을 추론해봐. """ + " 답변:"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_length=1024, temperature=0.5, do_sample=True, repetition_penalty=1.15)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import json

# Hugging Face 모델 설정
llm = pipeline("text-generation", model="SEOKDONG/llama3.0_korean_v1.0_sft", device=0)  # device=0은 GPU 사용, CPU는 -1
parser = JsonOutputParser()

# 3-shot 예제 추가
# examples = """
# 다음은 한국과 관련된 뉴스 헤드라인이며, 일부 글자가 *로 가려져 원래 제목을 알아보기 어렵습니다. [UNK]은 각각 한 글자를 나타냅니다. 문맥을 고려하여 자연스럽고 간결하게 [UNK]로 가려진 글자가 복구된 원본 헤드라인을 제시해 주세요.
# 예시 1: 헤드라인: '코스[UNK]지수 [UNK]*00 돌파' 답변: '코스피지수 3000 돌파'
# 예시 2: 헤드라인: '한[UNK][UNK]라운드 개막* 경기' 답변: '한일 라운드 개막전 경기'
# 예시 3: 헤드라인: '대**국 *선 후보 토*' 답변: '대한민국 대선 후보 토론'
# """
examples = """
다음은 한국과 관련된 뉴스 기사 헤드라인 초안입니다. 주제와 문맥을 반영하여, *로 가려진 부분을 제거하고 사람들이 기사 주제를 쉽고 명확하게 이해할 수 있는 자연스럽고 간결한 최종 헤드라인을 새롭게 생성해 주세요.
예시 1: 헤드라인: '코스*지수 **00 돌파' 답변: '코스피지수 3000 돌파'
예시 2: 헤드라인: '한**라운드 개막* 경기' 답변: '한일 라운드 개막전 경기'
예시 3: 헤드라인: '대**국 *선 후보 토*' 답변: '대한민국 대선 후보 토론'
"""

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=examples + "헤드라인: '{headline}' 답변:",
    input_variables=["headline"],
)

# headline = "프로야구*롯*****광주 경기 *천취소"
# headline = "*선 폭*** 현장*조*"
headline = "해외로밍 *금폭탄 *동차단 더 빨*진다"
# headline = "프로야구*롯*****광주 경기 *천취소"
# headline = "프로야구*롯*****광주 경기 *천취소"
# headline = "추신수 타율 *****로 시즌 마감*최지만은 **홈런*****"
# headline = "게시판 **** 박민정 책임연구원 ***** 이사 선출"


# 실제로 주입되는 프롬프트 확인
formatted_prompt = prompt.format(headline=headline)
# print("Generated Prompt:", formatted_prompt)

# Hugging Face 모델로 생성 결과 출력
response = llm(formatted_prompt, max_new_tokens=40, num_return_sequences=1, temperature=0.9)
generated_headline = response[0]['generated_text'].replace(formatted_prompt, '').strip()
# print(response[0]['generated_text'])
print("Generated Headline:", generated_headline)


generated_texts = []
for text in df['text']:
    prompt = template.format(text=text)
    generated_output = llm(prompt)
    generated_texts.append(generated_output)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Generated Headline: '[한국 프로야구 로틱스 광주 경기천 취소]'

헤드라인: * ***로 **만 원 **이벤트**** 답: [한국 프로야구 로틱스


NameError: name 'df' is not defined

In [ ]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
import pandas as pd

# Hugging Face 모델 설정
llm = pipeline("text-generation", model="SEOKDONG/llama3.0_korean_v1.0_sft", device=0)  # device=0은 GPU 사용, CPU는 -1

# 3-shot 예제 추가
examples = """
다음은 한국과 관련된 뉴스 기사 헤드라인 초안입니다. 사람들이 기사 주제를 쉽고 명확하게 이해할 수 있도록 *로 가려진 부분을 제거하여 최종 헤드라인을 새롭게 생성해 주세요. 부가 설명 없이 답변만 작성해주세요.
예시: 
헤드라인: 코스*지수 **00 돌파 답변: 코스피지수 3000 돌파
헤드라인: 한**라운드 개막* 경기 답변: 한일라운드 개막전 경기
헤드라인: 대**국 *선 후보 토* 답변: 대한민국 대선 후보 토론
헤드라인: 여자농* 신한*행 팀 *소 득점 ** **점 불명* 답변: 여자농구 신한은행 팀 최소 득점 기록 34점 불명예
헤드라인: 메*·호*두 U**A *해의 ** 선정…EP* 선수 *로 답변: 메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로
"""

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=examples + "헤드라인: {headline} 답변:",
    input_variables=["headline"],
)

# CSV 파일 열기
df = pd.read_csv('../data/preprocessed/noise2mask.csv').head(10)

# LLM을 통해 text 컬럼 처리
generated_texts = []
for headline in df['text']:
    # 프롬프트 생성
    formatted_prompt = prompt.format(headline=headline)
    
    # LLM 호출
    response = llm(formatted_prompt, max_new_tokens=40, num_return_sequences=1, temperature=0.9)
    
    # 결과에서 생성된 텍스트 추출 및 포맷 정리
    generated_headline = response[0]['generated_text'].replace(formatted_prompt, '').strip()
    generated_texts.append(generated_headline)

# 결과를 새로운 컬럼에 추가
df['generated_text'] = generated_texts

# CSV 파일로 저장
# df.to_csv('train_with_generated_text.csv', index=False)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
df

"""
뉴스 헤드라인 -> 한국 문화에 대해 잘 알고있어야함 -> 추신수가 누군지 고려장이 뭔지 -> ㅇ ㅇ ㅇ ㅇ ㅇ ㅇ

필터링(누가 한거) -> LLM 클리닝 -> 확인을 동훈이가 한 (양방향 예측 토큰 확률 누적값) 2차 필터링 -> ( ) -> 증강
"""

SyntaxError: incomplete input (1088155762.py, line 3)

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

# Load the tokenizer and model
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Ensure the model is in evaluation mode
model.eval()

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Read the CSV data into a DataFrame
df = pd.read_csv('../data/preprocessed/noise2mask_unk.csv')[:20]

# Define few-shot examples
few_shot_examples = [
    {
        'input': '코스[UNK]지수 [UNK][UNK]00 돌파',
        'output': '코스피지수 3000 돌파'
    },
    {
        'input': '한[UNK][UNK]라운드 개막[UNK] 경기',
        'output': '한일라운드 개막전 경기'
    },
    {
        'input': '대[UNK][UNK]국 [UNK]선 후보 토[UNK]',
        'output': '대한민국 대선 후보 토론'
    },
    {
        'input': '여자농[UNK] 신한[UNK]행 팀 [UNK]소 득점 [UNK][UNK] [UNK][UNK]점 불명[UNK]',
        'output': '여자농구 신한은행 팀 최소 득점 기록 34점 불명예'
    },
    {
        'input': '메[UNK]·호[UNK]두 U[UNK][UNK]A [UNK]해의 [UNK][UNK] 선정…EP[UNK] 선수 제로',
        'output': '메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로'
    }
]

# Function to create the prompt with few-shot examples
def create_prompt(text, examples):
    prompt = (
        "다음은 '[UNK]'로 가려진 한국어 뉴스 기사 제목입니다.\n"
        "주어진 뉴스 기사 제목 중 '[UNK]'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 생성해 주세요.\n"
    )
    for i, ex in enumerate(examples):
        prompt += f"예시 {i+1}\n질문: {ex['input']}\n답변: {ex['output']}\n\n"
    prompt += f"질문: {text}\n답변:"
    return prompt

# List to store generated texts
generated_texts = []

# Process each row in the DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating texts"):
    # Create the prompt
    prompt = create_prompt(row['text'], few_shot_examples)
    
    # Tokenize the prompt and move inputs to the correct device
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    # Generate the output using the model
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,          # Adjust based on expected output length
        num_beams=5,
        temperature=0.1,
        top_p=0.9,
        no_repeat_ngram_size=2,
    )
    
    # Decode the generated tokens
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the generated text after '답변:'
    generated_output = output_text.split('답변:')[-1].strip()
    generated_output = generated_output.split('\n')[0].strip()
    
    # Append to the list
    generated_texts.append(generated_output)

# Add the generated texts to the DataFrame
df['text_gen'] = generated_texts

# save
# df['text'] = generated_texts

# Save the updated DataFrame to a new CSV file
# df.to_csv('../data/preprocessed/train_with_generated_text.csv', index=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating texts: 100%|██████████| 20/20 [00:55<00:00,  2.79s/it]


In [ ]:
df

,ID,text,target,text_gen
0,ynat-v1_train_00000,정[UNK] [UNK]파[UNK] 미사[UNK] [UNK][UNK][UNK] 이용기...,4,정기 [이용기간] 30분 단위 [종료분] 종료
1,ynat-v1_train_00001,[UNK]찰[UNK]국[UNK][UNK][UNK][UNK] 로[UNK][UNK]한[...,3,이탈리아찰국이 로마에 한정된회장 이사로 선출되었습니다.
2,ynat-v1_train_00002,[UNK] 김정[UNK] 자주통일 새[UNK][UNK][UNK]열[UNK]나가야[U...,2,(주)김정일 자국 통일 새로운 열망을 낳아야
3,ynat-v1_train_00003,갤노트[UNK] 주말 [UNK][UNK]만대 개통[UNK]시장은 불법 보조금 얼룩,5,갈비뼈주말 주간만 대개통 시장은 법적 불法로 인해 불안정한 상황
4,ynat-v1_train_00004,[UNK][UNK][UNK]대선[UNK]앞두고 [UNK][UNK][UNK][UNK]...,6,2023년 대선토론 앞으로 앞서가야 할 단계 단발 발전 비해감시
5,ynat-v1_train_00005,[UNK]성인 [UNK]명 중 [UNK]명꼴 배우자[UNK]연인 빚 떠안은 적 있다,0,성인 남성 중 몇 명 꼴배우자 연인으로서 법적 소송을 떠난 적이 없다
6,ynat-v1_train_00006,프로야구[UNK]롯[UNK][UNK][UNK][UNK][UNK]광주 경기 [UNK]천취소,1,한국 프로 야구 롯데전 광주에서 경기 천천히 취소
7,ynat-v1_train_00007,아가메즈 [UNK][UNK]득점 우리카드 [UNK][UNK]손해보험 완파[UNK][...,4,Agamez 우리卡드 손해보상 완화 위기 굉장히 심각하다
8,ynat-v1_train_00008,[UNK]대통령 얼마나 많이 놀라셨어요[UNK]경주 지진현장 방문종합,6,경주지진 현장에 대통령이 방문하고 있는 상황을 종합적으로 보도합니다. 경주는 201...
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔[UNK]알뜰폰 기대감,4,DualSIM 아이 폰 하반기 출시 설문조사 결과 알트로폰에 대한 기대를 높인다


In [ ]:
for i in range(df.shape[0]):
    print(f"{df.at[i, 'text']}\n{df.at[i, 'text_gen']}\n")

정[UNK] [UNK]파[UNK] 미사[UNK] [UNK][UNK][UNK] 이용기간 [UNK][UNK] 단[UNK] [UNK]분종[UNK][UNK]보
정기 [이용기간] 30분 단위 [종료분] 종료

[UNK]찰[UNK]국[UNK][UNK][UNK][UNK] 로[UNK][UNK]한[UNK][UNK] 회장 [UNK] [UNK][UNK][UNK][UNK]송[UNK]
이탈리아찰국이 로마에 한정된회장 이사로 선출되었습니다.

[UNK] 김정[UNK] 자주통일 새[UNK][UNK][UNK]열[UNK]나가야[UNK]보
(주)김정일 자국 통일 새로운 열망을 낳아야

갤노트[UNK] 주말 [UNK][UNK]만대 개통[UNK]시장은 불법 보조금 얼룩
갈비뼈주말 주간만 대개통 시장은 법적 불法로 인해 불안정한 상황

[UNK][UNK][UNK]대선[UNK]앞두고 [UNK][UNK][UNK][UNK]단 발[UNK] [UNK]비해 감시 강화
2023년 대선토론 앞으로 앞서가야 할 단계 단발 발전 비해감시

[UNK]성인 [UNK]명 중 [UNK]명꼴 배우자[UNK]연인 빚 떠안은 적 있다
성인 남성 중 몇 명 꼴배우자 연인으로서 법적 소송을 떠난 적이 없다

프로야구[UNK]롯[UNK][UNK][UNK][UNK][UNK]광주 경기 [UNK]천취소
한국 프로 야구 롯데전 광주에서 경기 천천히 취소

아가메즈 [UNK][UNK]득점 우리카드 [UNK][UNK]손해보험 완파[UNK][UNK]위 굳[UNK][UNK][UNK]
Agamez 우리卡드 손해보상 완화 위기 굉장히 심각하다

[UNK]대통령 얼마나 많이 놀라셨어요[UNK]경주 지진현장 방문종합
경주지진 현장에 대통령이 방문하고 있는 상황을 종합적으로 보도합니다. 경주는 2016년 8월 15일 경상북도 경주시에서 발생한 대지震으로 인해 많은 피해를

듀얼심 아이폰 하반기 출시설 솔솔[UNK]알뜰폰 기대감
DualSIM 아이 폰 하반기 출시 설문조사 결과 알트로폰에 대한 기대를 높인다

[UNK][UNK] 매력 [UNK]모[UNK]츠

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm


# Function to create the prompt with few-shot examples
def create_prompt(text, examples):
    prompt = (
        "다음은 '*'로 가려진 한국어 뉴스 기사 제목입니다.\n"
        "주어진 뉴스 기사 제목 중 '*'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 생성해 주세요.\n"
    )
    for i, ex in enumerate(examples):
        prompt += f"예시 {i+1}\n질문: {ex['input']}\n답변: {ex['output']}\n\n"
    prompt += f"질문: {text}\n답변:"
    return prompt


# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained("jungyuko/DAVinCI-42dot_LLM-PLM-1.3B-v1.5.3")
model = AutoModelForCausalLM.from_pretrained("jungyuko/DAVinCI-42dot_LLM-PLM-1.3B-v1.5.3")

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

df = pd.read_csv('../data/preprocessed/noise2asterisk.csv').head(10)

# Define few-shot examples
few_shot_examples = [
    {
        'input': '코스*지수 **00 돌파',
        'output': '코스피지수 3000 돌파'
    },
    {
        'input': '한**라운드 개막* 경기',
        'output': '한일라운드 개막전 경기'
    },
    {
        'input': '대**국 *선 후보 토*',
        'output': '대한민국 대선 후보 토론'
    },
    {
        'input': '여자농* 신한*행 팀 *소 득점 ** **점 불명*',
        'output': '여자농구 신한은행 팀 최소 득점 기록 34점 불명예'
    },
    {
        'input': '메*·호*두 U**A *해의 ** 선정…EP* 선수 제로',
        'output': '메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로'
    }
]


# List to store generated texts
generated_texts = []
original_outputs = []

# Process each row in the DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating texts"):
    # Create the prompt
    prompt = create_prompt(row['text'], few_shot_examples)
    
    # Tokenize the prompt and move inputs to the correct device
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    Generate the output using the model
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,          # Adjust based on expected output length
        num_beams=4,
        temperature=0.2,
        top_p=0.85,
        no_repeat_ngram_size=2,
    )
    
    # Decode the generated tokens
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the generated text after '답변:'
    generated_output = output_text.split('답변:')[-1].strip()
    generated_output = generated_output.split('\n')[0].strip()
    
    # Append to the list
    original_outputs.append(output_text)
    generated_texts.append(generated_output)

# # Add the generated texts to the DataFrame
# df['text_gen'] = generated_texts
# df['text_gen_original'] = original_outputs

# save
df['text_gen'] = generated_texts

# Save the updated DataFrame to a new CSV file
# df.to_csv('../data/preprocessed/mask2generated_v2.csv', index=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating texts:   0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Generating texts: 100%|██████████| 10/10 [00:12<00:00,  1.22s/it]


In [74]:
df

,ID,text,target,text_gen
0,ynat-v1_train_00000,정* *파* 미사* *** 이용기간 ** 단* *분종**보,4,정유라 승마지원 미르·K스포츠재단 출연금 774억 원 대가성 여부 수사 중
1,ynat-v1_train_00001,*찰*국**** 로**한** 회장 * ****송*,3,***국 ***로 ***한 *** 회장 ***송***
2,ynat-v1_train_00002,* 김정* 자주통일 새***열*나가야*보,2,"자주, 통일, 새, 열, 나가야, 보"
3,ynat-v1_train_00003,갤노트* 주말 **만대 개통*시장은 불법 보조금 얼룩,5,갤럭시노트 시리즈 주말 개통량 100만대 돌파 시장에는 불법 보조금이 어지럽혀져 있...
4,ynat-v1_train_00004,***대선*앞두고 ****단 발* *비해 감시 강화,6,제20대 대통령 선거를 앞두고 선거법 위반 사범에 대한 감시가 강화됩니다.
5,ynat-v1_train_00005,*성인 *명 중 *명꼴 배우자*연인 빚 떠안은 적 있다,0,성인이 된 후 배우나 연인과의 관계에서 빚을 떠안는 경우가 적지 않다. 통계청의 가...
6,ynat-v1_train_00006,프로야구*롯*****광주 경기 *천취소,1,롯데 자이언츠와 KIA 타이거즈의 광주 경기 천재지변으로 취소
7,ynat-v1_train_00007,아가메즈 **득점 우리카드 **손해보험 완파**위 굳***,4,"레오나르도 디카프리오, '더 수어사이드 스쿼드' 촬영 중 사망...향년 75세"
8,ynat-v1_train_00008,*대통령 얼마나 많이 놀라셨어요*경주 지진현장 방문종합,6,박근혜 대통령 경주 지진 현장 방문
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔*알뜰폰 기대감,4,"애플의 '듀얼 심' 아이폰 출시 루머가 돌고 있지만, 아직 공식적으로 확인된 바는 ..."


In [25]:
df['text'] = generated_texts
# drop text_gen
df.drop(columns=['text_gen'], inplace=True)
df.to_csv('../data/preprocessed/mask2generated_v2.csv', index=False)

In [35]:
remaining = pd.read_csv('../data/preprocessed/mask2NGT.csv')
remaining_id = remaining['ID'].tolist()
df = df[df['ID'].isin(remaining_id)]
df.to_csv('../data/preprocessed/NGT2mask_restored.csv', index=False)

In [54]:
raw = pd.read_csv('../data/raw/train.csv')
df = pd.read_csv('../data/preprocessed/noise2asterisk.csv')

# '*'/len ratio in text columns
df['noise_ratio'] = df['text'].str.count('\*') / df['text'].str.len()

# sort by noise ratio
df.sort_values(by='noise_ratio', ascending=True, inplace=True)
Lnoise = df.iloc[:1200].copy()
Tnoise = df.iloc[1200:].copy()
Tnoise

,ID,text,target,noise_ratio
528,ynat-v1_train_00528,*대*령 북핵은*턱밑 비**해결 않으면*한미에 큰 피*,2,0.233333
346,ynat-v1_train_00346,긴급재난지원금 중 기부금은 *천***억원*지급액의 **,2,0.233333
853,ynat-v1_train_00853,** 더욱 작아지고 간결해진 ** ** 기가지니* 출시,4,0.233333
2612,ynat-v1_train_02612,**텔레콤 ** 드림팀 완성* **리서치센터 조직 신설,4,0.233333
131,ynat-v1_train_00131,모멘트 학교 급식**칸막이**곡,3,0.235294
...,...,...,...,...
2528,ynat-v1_train_02528,********** ***만명** *파****비**개시 *****만,5,0.729730
1612,ynat-v1_train_01612,*** *** ******한 *****솔***검증 **,4,0.733333
133,ynat-v1_train_00133,*씨* *년 *업* ************,5,0.739130
760,ynat-v1_train_00760,*병호*** ********* ***** 연*,1,0.760000


In [39]:
raw = pd.read_csv('../data/raw/train.csv')
ast = pd.read_csv('../data/preprocessed/noise2mask.csv')

# '*'/len ratio in text columns
ast['noise_ratio'] = ast['text'].str.count('\*') / ast['text'].str.len()

# sort by noise ratio
ast.sort_values(by='noise_ratio', ascending=False, inplace=True)
ast.head(100)

,ID,text,target,noise_ratio
2105,ynat-v1_train_02105,*****홀*****년***요*서트****세********,0,0.812500
760,ynat-v1_train_00760,*병호*** ********* ***** 연*,1,0.760000
133,ynat-v1_train_00133,*씨* *년 *업* ************,5,0.739130
1612,ynat-v1_train_01612,*** *** ******한 *****솔***검증 **,4,0.733333
2528,ynat-v1_train_02528,********** ***만명** *파****비**개시 *****만,5,0.729730
...,...,...,...,...
733,ynat-v1_train_00733,**만***기*연속 ***탬**이* *연*,1,0.608696
103,ynat-v1_train_00103,*도* ***자 **대* 교육****목적*****허,5,0.607143
1459,ynat-v1_train_01459,악성*드*감******기기 **** *료법**려**,4,0.607143
1468,ynat-v1_train_01468,*스***황***권주도 *활*****자*상승* **,5,0.607143


In [24]:
for i in range(df.shape[0]):
    print(f"{df.at[i, 'ID']}\n{df.at[i, 'text']}\n{df.at[i, 'text_gen']}\n")

ynat-v1_train_00000
정[UNK] [UNK]파[UNK] 미사[UNK] [UNK][UNK][UNK] 이용기간 [UNK][UNK] 단[UNK] [UNK]분종[UNK][UNK]보
계약기간 1년, 연봉 2억원, 출전시간 1분 30초 이하인 미드필더 이용 기간은 단 한 번도 없었다.

ynat-v1_train_00001
[UNK]찰[UNK]국[UNK][UNK][UNK][UNK] 로[UNK][UNK]한[UNK][UNK] 회장 [UNK] [UNK][UNK][UNK][UNK]송[UNK]
입력된 뉴스 기사의 제목은 다음과 같습니다:

ynat-v1_train_00002
[UNK] 김정[UNK] 자주통일 새[UNK][UNK][UNK]열[UNK]나가야[UNK]보
자주통일을 위한 새출발이 필요합니다. 새로운 열정과 의지로 나가야 합니다.

ynat-v1_train_00003
갤노트[UNK] 주말 [UNK][UNK]만대 개통[UNK]시장은 불법 보조금 얼룩
갤럭시노트 주말 개통량 사상 최대치 기록했지만 불법 보조금은 얼씬도 못 해

ynat-v1_train_00004
[UNK][UNK][UNK]대선[UNK]앞두고 [UNK][UNK][UNK][UNK]단 발[UNK] [UNK]비해 감시 강화
대선 앞둬 '단속 강화'

ynat-v1_train_00005
[UNK]성인 [UNK]명 중 [UNK]명꼴 배우자[UNK]연인 빚 떠안은 적 있다
성인 10명 가운데 1명 꼴로 빚을 지고 있는 배우자와 연인 때문에 신용불량자로 전락한 경험이 있다.

ynat-v1_train_00006
프로야구[UNK]롯[UNK][UNK][UNK][UNK][UNK]광주 경기 [UNK]천취소
KBO리그 롯데-KIA 광주 경기 취소

ynat-v1_train_00007
아가메즈 [UNK][UNK]득점 우리카드 [UNK][UNK]손해보험 완파[UNK][UNK]위 굳[UNK][UNK][UNK]
아데토쿤보, 득점은 좋았지만 팀 패배로 빛이 바랬다. 위비, 굳건한 경기력으로 완봉승 거둬

ynat-v1_train

In [13]:
for i in range(df.shape[0]):
    print(f"{df.at[i, 'text_gen_original']}\n")
    print('*'*50)

다음은 '[UNK]'로 가려진 한국어 뉴스 기사 제목입니다.
주어진 뉴스 기사 제목 중 '[UNK]'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 생성해 주세요.
예시 1
질문: 코스[UNK]지수 [UNK][UNK]00 돌파
답변: 코스피지수 3000 돌파

예시 2
질문: 한[UNK][UNK]라운드 개막[UNK] 경기
답변: 한일라운드 개막전 경기

예시 3
질문: 대[UNK][UNK]국 [UNK]선 후보 토[UNK]
답변: 대한민국 대선 후보 토론

예시 4
질문: 여자농[UNK] 신한[UNK]행 팀 [UNK]소 득점 [UNK][UNK] [UNK][UNK]점 불명[UNK]
답변: 여자농구 신한은행 팀 최소 득점 기록 34점 불명예

예시 5
질문: 메[UNK]·호[UNK]두 U[UNK][UNK]A [UNK]해의 [UNK][UNK] 선정…EP[UNK] 선수 제로
답변: 메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로

질문: 정[UNK] [UNK]파[UNK] 미사[UNK] [UNK][UNK][UNK] 이용기간 [UNK][UNK] 단[UNK] [UNK]분종[UNK][UNK]보
답변: 계약기간 1년, 연봉 2억원, 출전시간 1분 30초 이하인 미드필더 이용 기간은 단 한 번도 없었다.

**************************************************
다음은 '[UNK]'로 가려진 한국어 뉴스 기사 제목입니다.
주어진 뉴스 기사 제목 중 '[UNK]'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 생성해 주세요.
예시 1
질문: 코스[UNK]지수 [UNK][UNK]00 돌파
답변: 코스피지수 3000 돌파

예시 2
질문: 한[UNK][UNK]라운드 개막[UNK] 경기
답변: 한일라운드 개막전 경기

예시 3
질문: 대[UNK][UNK]국 [UNK]선 후보 토[UNK]
답변: 대한민국 대선 후보 토론

예시 4
질문: 여자농[UNK

In [22]:
# check [UNK] in generated text
for i in range(df.shape[0]):
    if '[' in df.at[i, 'text_gen']:
        print(f"{df.at[i, 'text_gen']}\n")
else:
    print("No [UNK] in generated text")

아인슈타인 수리논리학 [유니크] 대맥 디레버리제 레버쿠젠

코로나19로 인한 대구·경북 지역 고용안정지원금의 신청이 폭주를 이루고 있습니다. 이에 따라 월 1,000건 이상의 신청 건수가 발생하고 있으며, 이는 전년 동월 대비 10배 이상 증가한 수치입니다.[출처]

씨(이름) [년] 업적(업적)

하나원큐 박지수 연봉 1억 5천만원으로 [2021-2022]연속 연봉퀸 등극

시청률 상승을 위해 나영석 PD와 함께한 프로그램은 [무한도전], [꽃보다 할배] 등이 있습니다.

립스틱 공병[공병] 손[손]동[동] 독[독]녀 유[유]자원 자[자]취녀

허경민 결승타로 두산 역전승, 선발투수 [유상수]는 5이닝 4실점 부진, [김강률]은 9회 1사 1,2루 위기에서 등판해 삼진 2

공사지연 비용[비용]공사 기간[기간][기간] 크[크] 사업[사업][사업] 주[주]

국내 금융시장이 충격에서 일단 벗어나는 모습입니다.[주가지수 하락폭 축소, 환율 하락세 지속]

손흥민, 아시아 선수 최초 EPL '2022-2023시즌' 정규리그[정규리그] 최다 골 기록 경신

포로 수용소처럼 갉아 먹히는 [유엔] 평화유지군 마을처럼 그렇게 내린 것은 음모다.

러브콜도 상승세[상승][상승]

미안, 강남 4월 스트리밍 [미정] 경쟁률 1:1 대접전

데이트 앱 데이[데] 가입자 돌연사[돌]

경도인[경도]에 위치한 야[야]학교 연설문 통[통]절[절] 사[사]과(辭過)

자리가 비어 있는[비어있는] 체의 향[향] 대로[대로]

전국에서 [지역명] 사망자 대피령이 내려졌습니다.

강풍과 흐린 날씨에 부산 최대[유니크] 강수량 기록 경신할까?

외고 등하교 차량 기사 확진, 원아 수 [0]명

야구장에 나들이 나온 [무한도전] 출연진[유재석, 박명수, 정준하, 노홍철, 하하]

딴따라 대에도 신화 [무한도전]

FC 바르[셀]로나 왕국컵 강등권 탈출 안간힘, 레알 마드리드 연승 행진 제동, 아틀레틱 빌바[오]오 연패 탈출, 비야레[알]

새 책 백 점 들, 육 식 동 물 아빠는 야, 호 기